# 3.Text mining

Since biopython only work for python2 in my computer, before doing the following task, **please change the Kernel from python3 to python2.**<br>
1.Using Biopython, collect medline abstracts on "medulloblastoma" published in 2012. Save the data to disk as a CSV table.

In [40]:
# install biopython 
!pip show biopython
#!pip uninstall biopython
!pip list
import sys
print(sys.executable)

Name: biopython
Version: 1.76
Summary: Freely available tools for computational molecular biology.
Home-page: https://biopython.org/
Author: The Biopython Contributors
Author-email: biopython@biopython.org
License: UNKNOWN
Location: d:\users\punyung\anaconda3\envs\scripts\lib\site-packages
Requires: numpy
Required-by: 


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


Package         Version            
--------------- -------------------
biopython       1.76               
certifi         2019.11.28         
chardet         4.0.0              
idna            2.10               
numpy           1.16.6             
panda           0.3.1              
pandas          0.24.2             
pip             19.3.1             
python-dateutil 2.8.2              
pytz            2023.3             
requests        2.27.1             
setuptools      44.0.0.post20200106
six             1.16.0             
urllib3         1.26.16            
wheel           0.37.1             
wincertstore    0.2                
D:\Users\punyung\anaconda3\python.exe


DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [21]:
from Bio import Entrez
import csv
# Set the email address to be used for accessing the Entrez database
Entrez.email = 'punyung@163.com'
# Search for Medline abstracts on "medulloblastoma" published in 2012
search_term = 'medulloblastoma'
search_results = Entrez.read(Entrez.esearch(db='pubmed', term=search_term, datetype='pdat', mindate='2012/01/01', maxdate='2012/12/31'))

# Fetch the abstracts using the unique IDs obtained from the search results
id_list = search_results['IdList']
fetch_handle = Entrez.efetch(db='pubmed', id=id_list, rettype='abstract', retmode='text')
fetch_data = fetch_handle.read()
fetch_handle.close()

# Save the fetched data as a CSV table
csv_filename = 'medulloblastoma_abstracts_2012.csv'
#with open(csv_filename, 'w', encoding='utf-8') as csvfile:
with open(csv_filename, 'w') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Abstract'])
    for abstract in fetch_data.split('\n\n'):
        csv_writer.writerow([abstract])

2.Import the CSV table and build a SQLite database.

In [7]:
# Import the CSV data into a SQLite database
import sqlite3
from Bio import Entrez
import csv

db_filename = 'medulloblastoma_abstracts.db'
conn = sqlite3.connect(db_filename,timeout=10)
cursor = conn.cursor()
print ("database open successfully!")

database open successfully!


In [8]:
# Create a table in the database to store the abstracts
cursor.execute('CREATE TABLE IF NOT EXISTS abstracts (id INTEGER PRIMARY KEY AUTOINCREMENT, abstract TEXT)')

In [10]:
csv_filename = 'medulloblastoma_abstracts_2012.csv'
with open(csv_filename, 'r') as csvfile:
    csv_data = [row.decode('utf-8') for row in csvfile]
    
# Insert the CSV data into the database
for data_row in csv_data:
    cursor.execute('INSERT INTO abstracts (abstract)VALUES (?)', (data_row,))

In [11]:
cur = cursor.execute("SELECT abstract from abstracts")
for row in cur:
   print "ID = ", row[0]

ID =  Abstract

ID =  1. Small-molecule antagonists of Gli function.

ID =  "Ardecky R(1), Magnuson GK(1), Zou J(1), Ganji SR(1), Brown B(1), Ngo T(1), Lee 

ID =  J(1), Zeng FY(1), Sun Q(1), Stonich D(1), Salaniwal S(1), Sakata T(2), Rack 

ID =  PG(2), Casabar JKT(2), Mangravita-Novo A(3), Smith LH(3), Sergienko E(1), Chung 

ID =  TDY(1), Pinkerton AB(1), Pass I(1), Chen JK(2)."

ID =  "In: Probe Reports from the NIH Molecular Libraries Program [Internet]. Bethesda 

ID =  (MD): National Center for Biotechnology Information (US); 2010–.

ID =  2012 Dec 17 [updated 2014 May 13]."

ID =  "Author information:

ID =  (1)Sanford-Burnham Center for Chemical Genomics at Sanford-Burnham Medical 

ID =  Research Institute, La Jolla, California 92037, USA

ID =  (2)Department of Chemical and Systems Biology, Stanford University, Stanford, 

ID =  California 94305, USA.

ID =  (3)Sanford-Burnham Center for Chemical Genomics at Sanford-Burnham Medical 

ID =  Research Institute, Orlando, Florid

In [6]:
# Commit the changes and close the database connection
conn.commit()
conn.close()